In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import glob
from matplotlib import pylab as plt
from multiprocessing import Pool
import time
import cv2
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
type_dict = {"Normal":0, "Fire":1, "Water":2, "Electric":3, "Grass":4, "Ice":5,
    "Fighting":6, "Poison":7, "Ground":8, "Flying":9, "Psychic":10, "Bug":11,
    "Rock":12, "Ghost":13, "Dragon":14, "Dark":15, "Steel":16,"Fairy":17 }
def read_types(path):
    """タイプデータを読み込んで返す.

    渡されたパスからデータを読み込み，タイプデータのみを抽出，整形し返す.

    Args:
        path (str) : データパス

    Returns:
         Numpy arrays. タイプデータリストを二つ

    """
    print(f'\rloading types data ... ', end='')
    df = pd.read_csv(path+'Pokemon.csv', sep=',')
    df.drop_duplicates(subset='Number', inplace=True)
    df.reset_index(inplace=True, drop=True)
    ind = df[df['Type2'].isnull()]['Type2'].index
    df.iloc[ind, 3] = df.iloc[ind, 2]

    df_1 = df["Type1"][:801]
    df_1 = df_1.map(type_dict)
    type1 = df_1.values

    df_2 = df["Type2"][:801]
    df_2 = df_2.map(type_dict)
    type2 = df_2.values

    print(f'\rloading types data ... done')
    return type1, type2

In [3]:
type1,type2 = read_types('data/')
# print(type1)

loading types data ... done


In [4]:
from sklearn import svm, metrics, preprocessing, model_selection #機械学習用のライブラリを利用
from mlxtend.plotting import plot_decision_regions #学習結果をプロットする外部ライブラリを利用
import codecs
import dataset
from sklearn.model_selection import train_test_split
with codecs.open('data/convert_data//No_Mega_Weight_new.csv', 'r', 'utf-8', 'ignore') as f:
    df_pokemon_all = pd.read_csv(f)
df_pokemon_all


,Number,Name,Type1,Type2,Total,HP,Attack,Defense,SpecialAtk,SpecialDef,Speed,Generation,Legendary,weight
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,6.9
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,13.0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,100.0
3,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,8.5
4,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,878,Drakloak,Dragon,Ghost,410,68,80,50,60,50,102,8,False,11.0
878,879,Dragapult,Dragon,Ghost,600,68,80,50,60,50,102,8,False,50.0
879,880,Zacian,Fairy,Steel,720,92,170,115,80,115,148,8,True,355.0
880,881,Zamazenta,Fighting,Steel,720,92,130,145,80,145,128,8,True,785.0


In [5]:
Type1 = df_pokemon_all.iloc[:,2]
#Type_change = np.array(list(weight))

#change_li = Type_change.reshape(-1,1)
Type2 = df_pokemon_all.iloc[:,3]

In [6]:
import dataset_image
X = dataset_image.Data_set()
X1 = np.array(list(X))
X1 = X1.reshape(X1.shape[0], -1)
df = pd.DataFrame(X1)
df_change = pd.concat([df_pokemon_all, df],axis=1)
df_change
weight = df_change.iloc[:,13:27]
train_data = weight.values.tolist()
print(train_data)

Bulbasau [[124, 163, 141], [133, 162, 149], [122, 169, 133], [116, 157, 137]]
Ivysaur [[116, 111, 102], [86, 128, 125], [121, 141, 137], [105, 160, 172]]
Venusaur [[122, 99, 93], [96, 135, 133], [127, 100, 93], [90, 133, 133]]
Charmand [[178, 136, 110], [213, 191, 167], [56, 28, 25], [120, 86, 65]]
Charmele [[143, 83, 73], [173, 132, 120], [101, 52, 47], [155, 89, 77]]
Charizar [[75, 81, 72], [162, 123, 84], [86, 74, 56], [157, 131, 94]]
Squirtle [[126, 159, 166], [184, 184, 167], [55, 71, 76], [111, 125, 127]]
Wartortl [[161, 175, 202], [176, 173, 163], [104, 117, 128], [167, 177, 182]]
Blastois [[137, 118, 101], [125, 106, 95], [147, 140, 138], [127, 130, 140]]
Caterpie [[129, 175, 108], [190, 177, 119], [79, 104, 66], [88, 117, 72]]
Metapod [[133, 161, 94], [133, 166, 84], [28, 35, 18], [67, 86, 42]]
Butterfr [[72, 68, 72], [133, 120, 136], [192, 196, 200], [198, 204, 215]]
Weedle [[174, 132, 99], [139, 106, 76], [37, 29, 19], [5, 5, 5]]
Kakuna [[171, 157, 109], [150, 136, 95], [156

Primarin [[124, 132, 147], [176, 186, 204], [91, 124, 133], [78, 77, 78]]
Pikipek [[137, 106, 101], [63, 62, 62], [132, 124, 119], [90, 87, 84]]
Trumbeak [[49, 35, 28], [129, 127, 125], [84, 78, 74], [52, 53, 49]]
Toucanno [[103, 66, 46], [31, 31, 30], [128, 122, 107], [73, 67, 66]]
Yungoos [[103, 90, 78], [155, 123, 106], [43, 41, 39], [124, 99, 88]]
Gumshoos [[18, 15, 13], [56, 47, 36], [123, 104, 78], [159, 136, 97]]
Grubbin [[152, 132, 104], [150, 118, 86], [218, 173, 143], [158, 147, 128]]
Charjabu [[124, 155, 115], [173, 175, 166], [125, 155, 111], [99, 125, 99]]
Vikavolt [[106, 121, 137], [73, 81, 66], [106, 118, 132], [49, 53, 40]]
Crabrawl [[10, 10, 7], [42, 83, 101], [75, 111, 115], [113, 99, 114]]
Crabomin [[31, 32, 32], [116, 123, 129], [81, 84, 80], [143, 147, 157]]
Oricorio [[57, 27, 27], [134, 61, 64], [52, 21, 22], [74, 31, 34]]
Cutiefly [[202, 190, 153], [193, 184, 138], [217, 207, 193], [111, 107, 103]]
Riombee [[145, 137, 94], [113, 100, 74], [213, 205, 196], [147, 1

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import dataset_image

#print(X1)
train_X1 = train_data
train_y1 = Type1
(train_X1, test_X1 ,train_y1, test_y1) = train_test_split(train_X1, train_y1, test_size = 0.2, random_state = 666)
#print(train_X1)

In [8]:
clf1 = RandomForestClassifier(
    bootstrap=True, #ランダムにデータを使う。falseだと順番ずつデータを取る。日付とかを重要視するらしい。
    class_weight=None, #不均衡を考慮しないならNone,考慮するならclass_weight='balanced'
    criterion='gini', #スプリットの品質を測定する機能。
    max_depth=15, #最大の深さ
    max_features=10, 
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=1,
    min_samples_split=3,
    min_weight_fraction_leaf=0.0,
    n_estimators=300,
    n_jobs=1,
    oob_score=False,
    random_state=2525,
    verbose=0,
    warm_start=False)

clf1 = clf1.fit(train_X1, train_y1)
pred1 = clf1.predict(test_X1)
print(f"acc: {clf1.score(test_X1, test_y1)}")


acc: 0.22598870056497175


In [9]:
for predx,test_y in zip(pred1,test_y1):
    
    print(f"pred : {predx}------",f"test : {test_y}")


pred : Normal------ test : Rock
pred : Rock------ test : Steel
pred : Normal------ test : Dark
pred : Fire------ test : Fire
pred : Grass------ test : Ground
pred : Water------ test : Dragon
pred : Normal------ test : Water
pred : Bug------ test : Fire
pred : Electric------ test : Fire
pred : Bug------ test : Fire
pred : Water------ test : Normal
pred : Normal------ test : Normal
pred : Water------ test : Fighting
pred : Normal------ test : Water
pred : Bug------ test : Grass
pred : Grass------ test : Bug
pred : Normal------ test : Ground
pred : Normal------ test : Poison
pred : Bug------ test : Electric
pred : Normal------ test : Dark
pred : Water------ test : Normal
pred : Fire------ test : Fire
pred : Normal------ test : Fire
pred : Water------ test : Grass
pred : Water------ test : Bug
pred : Rock------ test : Grass
pred : Grass------ test : Normal
pred : Normal------ test : Psychic
pred : Normal------ test : Grass
pred : Normal------ test : Fairy
pred : Water------ test : Water
pr

In [10]:
#説明変数XとタイプのみのY分離
import pandas as pd
poke = pd.read_csv('data/convert_data/No_Mega_Weight_new.csv')
poke_noType = poke.drop(['Number','Type1','Type2','Total','HP','Attack','Defense','SpecialAtk','SpecialDef','Speed','Generation','Legendary'],axis=1)
X = []
Y = []
Y_type2 = []
l = len(poke)
col = poke_noType.columns.values
coll = len(col)
index = 0
while index < l:
    column = 0
    Xdata = []
    Y_nt = []
    while column < coll: 
        Xdata.append(poke_noType.iat[index,column])
        column += 1

    Y_nt.append(poke.at[index,'Type1'])
    if type(poke.at[index,'Type2']) is not type(poke.at[index,'Type1']):
        Y_nt.append(poke.at[index,'Type1'])
    else:
        Y_nt.append(poke.at[index,'Type2'])
    X.append(Xdata)
    Y.append(Y_nt)
    index += 1

for i in range(l):
    Y_type2.append(Y[i][1])

    
print(Y_type2)


['Poison', 'Poison', 'Poison', 'Fire', 'Fire', 'Flying', 'Water', 'Water', 'Water', 'Bug', 'Bug', 'Flying', 'Poison', 'Poison', 'Poison', 'Flying', 'Flying', 'Flying', 'Normal', 'Normal', 'Flying', 'Flying', 'Poison', 'Poison', 'Electric', 'Electric', 'Ground', 'Ground', 'Poison', 'Poison', 'Ground', 'Poison', 'Poison', 'Ground', 'Fairy', 'Fairy', 'Fire', 'Fire', 'Fairy', 'Fairy', 'Flying', 'Flying', 'Poison', 'Poison', 'Poison', 'Grass', 'Grass', 'Poison', 'Poison', 'Ground', 'Ground', 'Normal', 'Normal', 'Water', 'Water', 'Fighting', 'Fighting', 'Fire', 'Fire', 'Water', 'Water', 'Fighting', 'Psychic', 'Psychic', 'Psychic', 'Fighting', 'Fighting', 'Fighting', 'Poison', 'Poison', 'Poison', 'Poison', 'Poison', 'Ground', 'Ground', 'Ground', 'Fire', 'Fire', 'Psychic', 'Psychic', 'Steel', 'Steel', 'Flying', 'Flying', 'Flying', 'Water', 'Ice', 'Poison', 'Poison', 'Water', 'Ice', 'Poison', 'Poison', 'Poison', 'Ground', 'Psychic', 'Psychic', 'Water', 'Water', 'Electric', 'Electric', 'Psychic'

In [11]:
train_X2 = train_data
train_y2 = Y_type2
(train_X2, test_X2 ,train_y2, test_y2) = train_test_split(train_X2, train_y2, test_size = 0.2, random_state = 666)
#print(train_X1)
clf2 = RandomForestClassifier(
    bootstrap=True, #ランダムにデータを使う。falseだと順番ずつデータを取る。日付とかを重要視するらしい。
    class_weight=None, #不均衡を考慮しないならNone,考慮するならclass_weight='balanced'
    criterion='gini', #スプリットの品質を測定する機能。
    max_depth=15, #最大の深さ
    max_features=10, 
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=1,
    min_samples_split=3,
    min_weight_fraction_leaf=0.0,
    n_estimators=300,
    n_jobs=1,
    oob_score=False,
    random_state=2525,
    verbose=0,
    warm_start=False)

clf2 = clf2.fit(train_X2, train_y2)
pred2 = clf2.predict(test_X2)
print(f"acc: {clf2.score(test_X2, test_y2)}")

acc: 0.192090395480226


In [12]:
print(len(test_y1))
pred1_2 = clf1.predict(test_X2)
pred2_1 = clf2.predict(test_X1)


177


In [13]:
# print(len(pred1_2))
# print(len(test_y2))


In [14]:
j=0
for pred_1,testy1,pred_2,testy2,pred_12,pred_21 in zip(pred1,test_y1,pred2,test_y2,pred1_2,pred2_1):
    if pred_1 == testy1:
        j+=1
    elif pred_2 == testy2:
        j+=1
    elif pred_12 == testy2:
        j+=1
    elif pred_21 == testy1:
        j+=1

print(j)
score = j / 177
print(score)

64
0.3615819209039548


In [15]:
#グリッドサーチ込み
search_params = {
     'n_estimators'      : [5, 10, 20, 30, 50, 100, 300],
      'max_features'      : [3, 5, 10, 15, 20],
      'random_state'      : [2525],
      'n_jobs'            : [1],
      'min_samples_split' : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
      'max_depth'         : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100]
}

gs1 = GridSearchCV(RandomForestClassifier(),           # 対象の機械学習モデル
                  search_params,   # 探索パラメタ辞書
                  cv=5,            # クロスバリデーションの分割数
                  verbose=True,    # ログ表示
                  n_jobs=-1)       # 並列処理
gs1 = gs1.fit(train_X1, train_y1)
#gs2 = gs1.fit(train_X2, train_y2)
pred_gs1 = gs1.predict(test_X1)
#pred_gs2 = gs2.predict(test_X2)
print(f"acc: {gs1.score(test_X1, test_y1)}")

Fitting 5 folds for each of 3500 candidates, totalling 17500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   33.0s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   58.8s
[Parallel(n_jobs=-1)]: Done 1340 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2380 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 3204 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 4316 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 5702 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 6948 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 8062 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 9748 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 11534 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 13240 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done 15300 tasks     

acc: 0.21468926553672316


In [16]:
gs2 = gs1.fit(train_X2, train_y2)
pred_gs2 = gs2.predict(test_X2)
print(f"acc: {gs2.score(test_X2, test_y2)}")

Fitting 5 folds for each of 3500 candidates, totalling 17500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   58.3s
[Parallel(n_jobs=-1)]: Done 1380 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2480 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 3416 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 4244 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 5474 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 6456 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 7818 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 9240 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 10818 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 12700 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done 14494 tasks     

acc: 0.1751412429378531


In [19]:
pred_gs1_2 = gs1.predict(test_X2)
pred_gs2_1 = gs2.predict(test_X1)

In [20]:
js=0
for pred_1,testy1,pred_2,testy2,pred_12,pred_21 in zip(pred_gs1,test_y1,pred_gs2,test_y2,pred_gs1_2,pred_gs2_1):
    if pred_1 == testy1:
        js+=1
    elif pred_2 == testy2:
        js+=1
    elif pred_12 == testy2:
        js+=1
    elif pred_21 == testy1:
        js+=1

print(js)
score = js / 177
print(score)

56
0.3163841807909605


In [21]:
train_Xgen1 = train_data[:801]
train_ygen1 = Type1[:801]
train_ygen2 = Y_type2[:801]
#a_train, a_test = train_test_split(a, shuffle=False)
#(train_X2, test_X2 ,train_y2, test_y2) = train_test_split(, train_y2, test_size = 0.3, random_state = 666)
#print(train_X1)

test_Xgen1 = train_data[802:882]
test_ygen1 = Type1[802:882]
test_ygen2 = Y_type2[802:882]

In [33]:
print(train_Xgen1)
print(train_ygen2)

[[6.9, 124.0, 163.0, 141.0, 133.0, 162.0, 149.0, 122.0, 169.0, 133.0, 116.0, 157.0, 137.0], [13.0, 116.0, 111.0, 102.0, 86.0, 128.0, 125.0, 121.0, 141.0, 137.0, 105.0, 160.0, 172.0], [100.0, 122.0, 99.0, 93.0, 96.0, 135.0, 133.0, 127.0, 100.0, 93.0, 90.0, 133.0, 133.0], [8.5, 178.0, 136.0, 110.0, 213.0, 191.0, 167.0, 56.0, 28.0, 25.0, 120.0, 86.0, 65.0], [19.0, 143.0, 83.0, 73.0, 173.0, 132.0, 120.0, 101.0, 52.0, 47.0, 155.0, 89.0, 77.0], [90.5, 75.0, 81.0, 72.0, 162.0, 123.0, 84.0, 86.0, 74.0, 56.0, 157.0, 131.0, 94.0], [9.0, 126.0, 159.0, 166.0, 184.0, 184.0, 167.0, 55.0, 71.0, 76.0, 111.0, 125.0, 127.0], [22.5, 161.0, 175.0, 202.0, 176.0, 173.0, 163.0, 104.0, 117.0, 128.0, 167.0, 177.0, 182.0], [85.5, 137.0, 118.0, 101.0, 125.0, 106.0, 95.0, 147.0, 140.0, 138.0, 127.0, 130.0, 140.0], [2.9, 129.0, 175.0, 108.0, 190.0, 177.0, 119.0, 79.0, 104.0, 66.0, 88.0, 117.0, 72.0], [9.9, 133.0, 161.0, 94.0, 133.0, 166.0, 84.0, 28.0, 35.0, 18.0, 67.0, 86.0, 42.0], [32.0, 72.0, 68.0, 72.0, 133.0, 

In [23]:
# search_params = {
#     'max_depth'         : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
#     'max_features'      : [3, 5, 10, 15, 20],
#      'n_estimators'      : [5, 10, 20, 30, 50, 100, 300],
#       'random_state'      : [2525],
#       'n_jobs'            : [1],
#       'min_samples_split' : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
#     'min_impurity_decrease':[0.0]
# }
# gs1 = GridSearchCV(RandomForestClassifier(),
#                   search_params,
#                   cv=3,            # クロスバリデーションの分割数
#                   verbose=True,    # ログ表示
#                   n_jobs=-1)       # 並列処理

clf_gs1 = RandomForestClassifier(
    bootstrap=True, #ランダムにデータを使う。falseだと順番ずつデータを取る。日付とかを重要視するらしい。
    class_weight=None, #不均衡を考慮しないならNone,考慮するならclass_weight='balanced'
    criterion='gini', #スプリットの品質を測定する機能。
    max_depth=15, #最大の深さ
    max_features=10, 
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=1,
    min_samples_split=3,
    min_weight_fraction_leaf=0.0,
    n_estimators=300,
    n_jobs=1,
    oob_score=False,
    random_state=2525,
    verbose=0,
    warm_start=False)

clf_gs1 = clf_gs1.fit(train_Xgen1, train_ygen1)
pred_gs1 = clf_gs1.predict(test_Xgen1)
print(f"acc: {clf_gs1.score(test_Xgen1, test_ygen1)}")



acc: 0.1125


In [24]:
clf_gs2 = RandomForestClassifier(
    bootstrap=True, #ランダムにデータを使う。falseだと順番ずつデータを取る。日付とかを重要視するらしい。
    class_weight=None, #不均衡を考慮しないならNone,考慮するならclass_weight='balanced'
    criterion='gini', #スプリットの品質を測定する機能。
    max_depth=15, #最大の深さ
    max_features=10, 
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=1,
    min_samples_split=3,
    min_weight_fraction_leaf=0.0,
    n_estimators=300,
    n_jobs=1,
    oob_score=False,
    random_state=2525,
    verbose=0,
    warm_start=False)

clf_gs2 = clf_gs2.fit(train_Xgen1, train_ygen2)
pred_gs2 = clf_gs2.predict(test_Xgen1)
print(f"acc: {clf_gs2.score(test_Xgen1, test_ygen2)}")

acc: 0.125


In [25]:
# gs2 = GridSearchCV(RandomForestClassifier(),
#                   search_params,
#                   cv=3,            # クロスバリデーションの分割数
#                   verbose=True,    # ログ表示
#                   n_jobs=-1) 
# gs2 = gs2.fit(train_Xgen1,train_ygen2)

In [26]:
print(len(pred_gs2))
print(len(pred_gs1))
pred_gs1 = clf_gs1.predict(test_Xgen1)

80
80


In [30]:
# print(gs1.score(test_Xgen1,test_ygen1))
# print(gs2.score(test_Xgen1,test_ygen2))

#print(f"acc: {gs2.score(train_Xgen2_test, train_ygen2_test2)}")
jj = 0
for gs1,test1,gs2,test2 in zip(pred_gs1 ,test_ygen1,pred_gs2,test_ygen2):
    if gs1 == test1:
        jj+=1
    elif gs2 == test2:
        jj+=1
    elif gs2 == test1:
        jj+=1
    elif gs1 == test2:
        jj+=1
print(jj)
score = jj / 80
print(score)

18
0.225


In [35]:
gs_gs = GridSearchCV(RandomForestClassifier(),           # 対象の機械学習モデル
                  search_params,   # 探索パラメタ辞書
                  cv=5,            # クロスバリデーションの分割数
                  verbose=True,    # ログ表示
                  n_jobs=-1)       # 並列処理
gs_gs1 = gs_gs.fit(train_Xgen1, train_ygen1)
pred_gs_gs1 = gs_gs1.predict(test_Xgen1)
print(f"acc: {gs_gs1.score(test_Xgen1, test_ygen1)}")


Fitting 5 folds for each of 3500 candidates, totalling 17500 fits


/Users/e175752/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1372 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 2168 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 2818 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 4100 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 5260 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 6244 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 7650 tasks    

acc: 0.1125


[Parallel(n_jobs=-1)]: Done 17500 out of 17500 | elapsed: 21.2min finished


In [36]:
gs_gs2 = gs_gs.fit(train_Xgen1, train_ygen2)
pred_gs_gs2 = gs_gs2.predict(test_Xgen1)
print(f"acc: {gs_gs1.score(test_Xgen1, test_ygen2)}")

Fitting 5 folds for each of 3500 candidates, totalling 17500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1848 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 2834 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 3584 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 4434 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 5500 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 6550 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 7700 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 9198 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 10548 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 11998 tasks      

acc: 0.1125


In [37]:
jjs = 0
for gs1,test1,gs2,test2 in zip(pred_gs_gs1 ,test_ygen1,pred_gs_gs2,test_ygen2):
    if gs1 == test1:
        jjs+=1
    elif gs2 == test2:
        jjs+=1
    elif gs2 == test1:
        jjs+=1
    elif gs1 == test2:
        jjs+=1
print(jjs)
score = jjs / 80
print(score)

17
0.2125


In [40]:
#print(gs1.best_estimator_)
#print(gs2.best_estimator_)
print(gs_gs1.best_estimator_)
print(gs_gs2.best_estimator_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features=5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=30,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                       oob_score=False, random_state=2525, verbose=0,
                       warm_start=False)
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features=5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=30,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                       oob_score=Fa